## Importación de librerías

In [66]:
import pandas as pd
import os
import glob
import csv
import json

## Configuración de variables globales

In [67]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
DATASETS_PATH = SETUP_JSON['datasets_path'] # Path to the datasets,
DATASETS_FOLDER = os.path.join(os.getcwd(), DATASETS_PATH) # Folder containing the datasets,
DATASETS = glob.glob(os.path.join(DATASETS_FOLDER, '*.csv')) # List of datasets
OUTPUT_CSV = SETUP_JSON['output_csv'] # Output CSV file
OUTPUT_PARQUET = SETUP_JSON['output_parquet'] # Output CSV file
N_ROWS = SETUP_JSON['n_rows']

## Obtener todas las características diferentes

In [68]:
datasets = [f"{DATASETS_PATH}\\02-14-2018.csv", f"{DATASETS_PATH}\\02-15-2018.csv", f"{DATASETS_PATH}\\02-16-2018.csv", f"{DATASETS_PATH}\\02-21-2018.csv"]

In [69]:
unique_headers = set()
for dataset in DATASETS:
    df = pd.read_csv(dataset, nrows=1)
    df_dtypes = df.dtypes.keys()
    unique_headers.add(tuple(df_dtypes))
for header in unique_headers:
    print(header)

('Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'F

In [70]:
headers = {}
for header in unique_headers:
    datasets = []
    for dataset in DATASETS:
        df = pd.read_csv(dataset, nrows=1)
        df_dtypes = df.dtypes.keys()
        if tuple(df_dtypes) == header:
            datasets.append(dataset)
    headers[header] = datasets
print(headers)

{('Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', '

In [71]:
def find_datasets_with_same_header():
    headers = {}
    same_files = {}

    for file in DATASETS:
        with open(file, newline='', encoding='utf-8') as f:
            header = tuple(next(csv.reader(f), None))
            if header:
                if header in headers:
                    headers[header].append(file)
                else:
                    headers[header] = [file]
    print("Archivos con headers distintos:", tuple(headers.values())[1])
    return tuple(headers.values())[0]

DATASETS = find_datasets_with_same_header()

Archivos con headers distintos: ['c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-20-2018.csv']


In [72]:
def mixed_types_columns(df):
    """
    Recorre todas las columnas del DataFrame y devuelve un diccionario
    con las columnas que contienen más de un tipo de dato.

    Args:
        df (pd.DataFrame): El DataFrame a analizar.

    Returns:
        dict: Un diccionario con nombres de columnas como claves,
              y otro diccionario con los tipos de datos y sus cantidades como valores.
    """
    mixed_types_columns = {}

    for col in df.columns:
        types = {}
        for val in df[col]:
            typeVal = type(val)
            types[typeVal] = types.get(typeVal, 0) + 1

        if len(types) > 1:
            mixed_types_columns[col] = types

    return mixed_types_columns

df = pd.read_csv('merged_output.csv')
mixed_types_columns = mixed_types_columns(df)
for col, types in mixed_types_columns.items():
    print(f"Columna: {col}")
    for typeVal, quantity in types.items():
        print(f"  type: {typeVal.__name__}, Cantidad: {quantity}")
    print()

C:\Users\isard\AppData\Local\Temp\ipykernel_3528\661668898.py:26: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_output.csv')


TypeError: 'numpy.dtypes.Int64DType' object is not callable

In [ ]:
def comparar_tipos_en_columnas(datasets):
    """
    Compara los tipos de datos presentes en columnas comunes entre varios datasets.

    Args:
        datasets (list): Lista de rutas a archivos CSV o DataFrames.

    Returns:
        dict: Un diccionario con nombres de columnas como claves.
              Cada valor es una lista con un resumen de los tipos múltiples detectados por dataset.
    """
    columnas_con_tipos_multiples = {}

    for i, ds in enumerate(datasets):
        # Cargar dataset si es ruta, o usar directamente si es DataFrame
        df = pd.read_csv(ds) if isinstance(ds, str) else ds

        for columna in df.columns:
            tipos = {}
            for val in df[columna]:
                tipo = type(val)
                tipos[tipo] = tipos.get(tipo, 0) + 1

            if len(tipos) > 1:
                if columna not in columnas_con_tipos_multiples:
                    columnas_con_tipos_multiples[columna] = []
                columnas_con_tipos_multiples[columna].append({
                    'dataset_index': i,
                    'tipos': {t.__name__: c for t, c in tipos.items()}
                })

    return columnas_con_tipos_multiples

columnas_multiples = comparar_tipos_en_columnas(DATASETS)
for columna, problemas in columnas_multiples.items():
    print(f"Columna: {columna}")
    for problema in problemas:
        print(f"  Dataset #{problema['dataset_index']}:")
        for tipo, cantidad in problema['tipos'].items():
            print(f"    Tipo: {tipo}, Cantidad: {cantidad}")
    print()


C:\Users\isard\AppData\Local\Temp\ipykernel_3528\2725541326.py:16: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ds) if isinstance(ds, str) else ds
C:\Users\isard\AppData\Local\Temp\ipykernel_3528\2725541326.py:16: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ds) if isinstance(ds, str) else ds
C:\Users\isard\AppData\Local\Temp\ipykernel_3528\2725541326.py:16: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,1

Columna: Dst Port
  Dataset #2:
    Tipo: int, Cantidad: 1040383
    Tipo: str, Cantidad: 8192
  Dataset #6:
    Tipo: int, Cantidad: 376832
    Tipo: str, Cantidad: 236272
  Dataset #7:
    Tipo: str, Cantidad: 60789
    Tipo: int, Cantidad: 270336

Columna: Protocol
  Dataset #2:
    Tipo: int, Cantidad: 1040383
    Tipo: str, Cantidad: 8192
  Dataset #6:
    Tipo: int, Cantidad: 376832
    Tipo: str, Cantidad: 236272
  Dataset #7:
    Tipo: str, Cantidad: 60789
    Tipo: int, Cantidad: 270336

Columna: Flow Duration
  Dataset #2:
    Tipo: int, Cantidad: 1040383
    Tipo: str, Cantidad: 8192
  Dataset #6:
    Tipo: int, Cantidad: 376832
    Tipo: str, Cantidad: 236272
  Dataset #7:
    Tipo: str, Cantidad: 60789
    Tipo: int, Cantidad: 270336

Columna: Tot Fwd Pkts
  Dataset #2:
    Tipo: int, Cantidad: 1040383
    Tipo: str, Cantidad: 8192
  Dataset #6:
    Tipo: int, Cantidad: 376832
    Tipo: str, Cantidad: 236272
  Dataset #7:
    Tipo: str, Cantidad: 60789
    Tipo: int, Canti

In [ ]:
columns_lengths = []

# Obtener número de columnas por dataset
for dataset in DATASETS:
    columns_length = len(pd.read_csv(dataset, nrows=1).dtypes)
    columns_lengths.append(columns_length)

# Obtener grupo de características más largo
for dataset in DATASETS:
    df = pd.read_csv(dataset, nrows=1)
    columns_length = len(df.dtypes)
    if (columns_length == max(columns_lengths)):
        X_columns = df.dtypes.to_dict()
        X_columns.pop('Label')
print(len(X_columns))
for column, type in X_columns.items():
    print(column, type)

79
Dst Port int64
Protocol int64
Timestamp object
Flow Duration int64
Tot Fwd Pkts int64
Tot Bwd Pkts int64
TotLen Fwd Pkts int64
TotLen Bwd Pkts int64
Fwd Pkt Len Max int64
Fwd Pkt Len Min int64
Fwd Pkt Len Mean float64
Fwd Pkt Len Std float64
Bwd Pkt Len Max int64
Bwd Pkt Len Min int64
Bwd Pkt Len Mean int64
Bwd Pkt Len Std float64
Flow Byts/s float64
Flow Pkts/s float64
Flow IAT Mean float64
Flow IAT Std float64
Flow IAT Max int64
Flow IAT Min int64
Fwd IAT Tot int64
Fwd IAT Mean float64
Fwd IAT Std float64
Fwd IAT Max int64
Fwd IAT Min int64
Bwd IAT Tot int64
Bwd IAT Mean float64
Bwd IAT Std float64
Bwd IAT Max int64
Bwd IAT Min int64
Fwd PSH Flags int64
Bwd PSH Flags int64
Fwd URG Flags int64
Bwd URG Flags int64
Fwd Header Len int64
Bwd Header Len int64
Fwd Pkts/s float64
Bwd Pkts/s float64
Pkt Len Min int64
Pkt Len Max int64
Pkt Len Mean float64
Pkt Len Std float64
Pkt Len Var float64
FIN Flag Cnt int64
SYN Flag Cnt int64
RST Flag Cnt int64
PSH Flag Cnt int64
ACK Flag Cnt int64
U

## Combinar datasets

Unimos todos los archivos CSV en un sólo dataframe y lo guardamos en formato CSV (para analizarlo con Tableau) y en formato Parquet (para su tratamiento con Python).

In [ ]:
df = pd.DataFrame()
for dataset in DATASETS:
    # Obtener las 10000 primeras filas del dataset
    current_df = pd.read_csv(dataset, nrows=N_ROWS, dtype=str) # Leer en formato string
    # Añadir columnas faltantes
    for column in X_columns.keys():
        if column not in current_df.columns:
            current_df[column] = 0 # Rellenar columnas con ceros
    print(f"Dataset {dataset} procesado: {current_df.shape})")
    # Concatenar dataframe actual con el dataframe final
    df = pd.concat([df, current_df], ignore_index=True, sort=False)

Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv procesado: (50000, 80))
Dataset c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv procesado: (50000, 80))


## Exportación de archivos JSON

Exportar todas las columnas y sus tipos 

In [ ]:
"""X_columns = {col: str(dtype) for col, dtype in X_columns.items()}

# Guardar en JSON
with open('X_columns.json', 'w') as f:
    json.dump(X_columns, f, indent=4)"""

"X_columns = {col: str(dtype) for col, dtype in X_columns.items()}\n\n# Guardar en JSON\nwith open('X_columns.json', 'w') as f:\n    json.dump(X_columns, f, indent=4)"

In [ ]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0,0,0,0,56320859.5,139.3000358938,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0,0,0,0,56320733,114.5512985522,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0,0,0,0,56319311.5,301.9345955667,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0,0,0,0,0,0,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0,0,0,0,0,0,0,0,Benign


In [ ]:
# Obtener todas las etiquetas únicas del dataframe

labels_list = list(df['Label'].unique())
print(labels_list.remove('Label'))


# Asignar índice a cada etiqueta
labels = {}
for i, label in enumerate(labels_list):
    labels[label] = i

# Guardar etiquetas en JSON
with open('labels.json', 'w') as f:
    json.dump(labels, f, indent=4)

None


## Exportación del dataframe en CSV y Parquet

In [ ]:
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0,0,0,0,56320859.5,139.3000358938,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0,0,0,0,56320733,114.5512985522,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0,0,0,0,56319311.5,301.9345955667,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0,0,0,0,0,0,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0,0,0,0,0,0,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,8080,6,02/03/2018 11:30:58,9722,3,4,326,129,326,0,...,20,0,0,0,0,0,0,0,0,Bot
449996,8080,6,02/03/2018 11:30:58,525,2,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Bot
449997,8080,6,02/03/2018 11:30:58,12784,3,4,326,129,326,0,...,20,0,0,0,0,0,0,0,0,Bot
449998,8080,6,02/03/2018 11:30:58,708,2,0,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Bot


In [ ]:
# Guardar en CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Archivo CSV guardado como {OUTPUT_CSV}")

# Guardar en Parquet
df.to_parquet(OUTPUT_PARQUET, index=False)
print(f"Archivo Parquet guardado como {OUTPUT_PARQUET}")

Archivo CSV guardado como merged_output.csv
Archivo Parquet guardado como merged_output.parquet


In [ ]:

def find_datasets_with_same_header():
    headers = {}
    same_files = {}

    for file in DATASETS:
        with open(file, newline='', encoding='utf-8') as f:
            header = tuple(next(csv.reader(f), None))
            if header:
                if header in headers:
                    headers[header].append("datasets\\"+file)
                else:
                    headers[header] = ["datasets\\"+file]
    print("Archivos con headers distintos:", tuple(headers.values())[1])
    return tuple(headers.values())[0]